# SMAP-USDA monthly averages

James McCreight 
Josh Daniel

Monthly climatologies/stats of soil moisture quantities and 
their derivatives (porosity and available porosity).

Moativating Question: What is the additional amount of water that soil could hold?

Notes about the data:
* This product is poorly documented but this appears to be a mostly a
  modeled/assimilated product similar to 
  https://ieeexplore.ieee.org/document/5353747 
  which used AMSR-E instead of SMAP.
* IPAD is the 2-layer soil moisture model used 
* "For this  study,  the  soil moisture  estimates  from  AMSR-E  are assumed
   to represent a soil depth comparable to the surface layer used by the IPAD 
   modified Palmer model."
* "The available soil moisture capacity in for the soil column is calculated 
  from the available water content (AWC) of both layers (i.e., AWC_sz and AWC_rz
  for the surface and root-zone model layers respectively) derived from  the  
  FAO  Digital  Soil  Map  of  the World [15], soil texture and the total 
  depth of the soil column.
* For compatibility with the volumetric soil moisture retrievals from AMSR-E, 
  we trans-formed  soil  moisture  units  from  depth  to  volumetric  content
  during  implementation  of  our  data  assimilation  system  (Sec-tion  IV)
  using  soil  porosity  data  from  the  Soil  Survey  Geo-graphic (SSURGO) database. 
* 1-D EnKF assimilates surface obs into IPAD (error covariances in the vertical?)

Bands (some interpretation):  
`smp`: soil moisture profile (fraction - volumetric?), range=[0,1], assimilated  
`ssm`: surface soil moisture (mm), range=[0, 25.39], observed  
`susm`: subsurface soil moisture (mm), range=[0, 274.6], assimilated  

Then:  
```
smp = (ssm + susm) / porosity_mm  [ - = mm/mm ]
porosity_mm = (ssm + susm) / smp
```
where porosity (SSURGO) is total  column (variable, per FAO).
So the unused porosity or additional porosity:

```
additional_porosity = porosity_mm - (ssm + susm)
  = (ssm + susm)/smp - (ssm + susm)
  = (ssm + susm - smp*ssm -smp*susm)/smp
  = (ssm(1 - smp) + susm(1 - smp)) / smp
  = (1 - smp)(ssm + susm)/smp
```

In [1]:
import mon_stats
import ee

In [2]:
#ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
smap_usda = (ee
  .ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture'))

In [5]:
smap_usda_clim = mon_stats.bands_avgs(['smp', 'ssm', 'susm'], smap_usda)

In [6]:
smap_usda_clim

{'avgs': <ee.dictionary.Dictionary at 0x7f930085d8b0>,
 'lens': <ee.dictionary.Dictionary at 0x7f930085daf0>}

In [7]:
smap_usda_clim['avgs'].getInfo()

{'smp': {'type': 'ImageCollection', 'bands': []},
 'ssm': {'type': 'ImageCollection', 'bands': []},
 'susm': {'type': 'ImageCollection', 'bands': []}}

In [8]:
smap_usda_clim['lens'].getInfo()

{'smp': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 61],
 'ssm': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 61],
 'susm': [51, 45, 51, 60, 61, 60, 64, 61, 60, 60, 59, 61]}

In [9]:
# separate the variables
smp = ee.ImageCollection(smap_usda_clim['avgs'].get('smp'))
ssm = ee.ImageCollection(smap_usda_clim['avgs'].get('ssm'))
susm = ee.ImageCollection(smap_usda_clim['avgs'].get('susm'))

In [10]:
months = mon_stats.months_list()

In [11]:
# porosity in mm (millimeters)
def mon_porosity_calc(mm):
  porosity = (ssm
    .filter(ee.Filter.eq('month', mm)).first()
    .add(susm.filter(ee.Filter.eq('month', mm)).first())
    .divide(smp.filter(ee.Filter.eq('month', mm)).first())
    .set('month', mm)
    .set('variable', 'porosity_mm')
    .rename(['porosity_mm']))
  return ee.Image(porosity)

porosity = ee.ImageCollection(months.map(mon_porosity_calc))
# porosity.getInfo()

In [12]:
# available/additional/unused porosity
def mon_avail_porosity_calc(mm):
  avail_porosity = (porosity
    .filter(ee.Filter.eq('month', mm)).first()
    .subtract(
      ssm.filter(ee.Filter.eq('month', mm)).first()
      .add(susm.filter(ee.Filter.eq('month', mm)).first()))
      .set('month', mm)
      .set('band', 'avail_porosity_mm')
      .rename(['avail_porosity_mm']))
  return ee.Image(avail_porosity)

avail_porosity = ee.ImageCollection(months.map(mon_avail_porosity_calc))
# avail_porosity.getInfo()

In [13]:
# Export to images, build a collection
smap_collection = ee.ImageCollection(
  ee.List([
    ssm.toList(12),
    susm.toList(12),
    smp.toList(12),
    porosity.toList(12),
    avail_porosity.toList(12)]).flatten())

# smap_collection.getInfo()

In [14]:
smap_asset = ee.data.createAsset({'type': 'ImageCollection'}, 'users/jamesmcc/smap_usda_climatology')
#smap_asset = {'id': 'users/jamesmcc/smap_usda_climatology_2'}

In [15]:
smap_asset

{'type': 'IMAGE_COLLECTION',
 'name': 'projects/earthengine-legacy/assets/users/jamesmcc/smap_usda_climatology',
 'id': 'users/jamesmcc/smap_usda_climatology',
 'updateTime': '2021-01-14T23:36:31.584919Z'}

In [16]:
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
bands = ['smp', 'ssm', 'susm', 'porosity_mm', 'avail_porosity_mm']
bbox = ee.Geometry.BBox(-180.0, -90, 180, 90)
for bb in bands:
    for mm0 in range(12):
        mm = mm0 + 1
        description = 'SMAP-USDA_mon_clim_' + str(bb) + '_' + str(mm).zfill(2) + '_' + month_names[mm0]
        xx = ee.batch.Export.image.toAsset(
            smap_collection
            .filter(ee.Filter.eq('month', mm))
            .filter(ee.Filter.eq('band', bb)).first(),
            description=description, 
            assetId=smap_asset['id'] + '/' + description,
            region=bbox).start()